Will base it off this hugging face tutorial https://colab.research.google.com/drive/14PtqRbrGa70M5NJnLfSllyJYyvl1nYnK?usp=sharing

# First format the manually labelled testing and training data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from os.path import exists
import numpy as np
import pandas as pd

#convert labelled excel data to array
df = pd.read_excel('/content/drive/MyDrive/Saved-You-A-Click-CS224N/data/title_classification/data_label_qs.xlsx')
df=df[0:400] #only first 400 are labelled
dnp = df.to_numpy()
ind=0
# Let 'Question' titles be 1 and 'Summary' titles be 0
for entry in dnp[:,2]:
    if entry == 'Q':
        dnp[:,2][ind] = 1
    else:
      # everything that is not Q is S, L or B. L is for list answer, and B is for bad title.
      # The no. of L and B < 10, so let us just let these be S/ Summary problems too.
        dnp[:,2][ind] = 0
    ind+=1
np.save('/content/drive/MyDrive/Saved-You-A-Click-CS224N/data/title_classification/datalabel_for_classification_numpy.npy', dnp)
df = pd.DataFrame(dnp, columns = ['titles','answer','QorS'])
df.to_pickle("/content/drive/MyDrive/Saved-You-A-Click-CS224N/data/title_classification/datalabel_for_classification_pandas.pkl")

The format of the data is [Clickbait title, Reddit User Answer, 1 (question) or 0 (summary)]. Though we only need the title and the 1 and 0's.



In [3]:
# Load to double check
dnp = np.load('/content/drive/MyDrive/Saved-You-A-Click-CS224N/data/title_classification/datalabel_for_classification_numpy.npy', allow_pickle=True)
import pickle
with open('/content/drive/MyDrive/Saved-You-A-Click-CS224N/data/title_classification/datalabel_for_classification_pandas.pkl', 'rb') as f:
    df = pickle.load(f)

Now let us put it into the format for DistillBERT

In [4]:
# lets make 70% of the 400 entries training, and 30% test data
train = dnp[0:280]
test = dnp[280:400]

X_train = pd.DataFrame(train[:,0])
X_test =pd.DataFrame(test[:,0])
y_train = pd.DataFrame(train[:,2])
y_test = pd.DataFrame(test[:,2])

pd.DataFrame(X_train)[:3]

,0
0,What Getting Rid of the Electoral College woul...
1,This Couple Waited 10 Years Before Opening The...
2,After Girl’s Dad Cuts Her Hair Off For Getting...


## check the shapes and split proportion 

In [5]:
X_train.shape, X_test.shape, y_train.shape

((280, 1), (120, 1), (280, 1))

In [6]:
print('The proportion in y_train\n',y_train.value_counts(normalize=True).mul(100))
print('The proportion in y_test\n',y_test.value_counts(normalize=True).mul(100))

The proportion in y_train
 1    64.642857
0    35.357143
dtype: float64
The proportion in y_test
 1    71.666667
0    28.333333
dtype: float64


## Preprocess

## Encode with  DistilBertTokenizer

import and installs

In [7]:
!pip install transformers

import pandas as pd
import tensorflow as tf
import transformers
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification

pd.set_option('display.max_colwidth', None)
MODEL_NAME = 'distilbert-base-uncased-finetuned-sst-2-english'# This model is a fine-tune checkpoint of DistilBERT-base-uncased, fine-tuned on SST-2.
BATCH_SIZE = 16
N_EPOCHS = 3 # we can put more, because evaluation of the model shows big difference in loss with accuracy 1.0

     |████████████████████████████████| 3.5 MB 9.8 MB/s 
     |████████████████████████████████| 596 kB 31.8 MB/s 
     |████████████████████████████████| 6.8 MB 22.6 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 895 kB 34.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Tokenizer

In [8]:
#define a tokenizer object
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)

#tokenize the text (padding to max sequence in batch)
train_encodings = tokenizer(list([x[0] for x in X_train.values]), truncation=True, padding=True)
test_encodings = tokenizer(list([x[0] for x in X_test.values]), truncation=True, padding=True)

#print the first paragraph and it transformation
print(f'First paragraph: \'{X_train[:1]}\'')
print(f'Input ids: {train_encodings["input_ids"][0]}')
print(f'Attention mask: {train_encodings["attention_mask"][0]}')

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

First paragraph: '                                                             0
0  What Getting Rid of the Electoral College would actually do'
Input ids: [101, 2054, 2893, 9436, 1997, 1996, 6092, 2267, 2052, 2941, 2079, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Attention mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


length check

In [9]:
len(train_encodings["attention_mask"][0]) #max len tokenized sentence - 362

49

In [10]:
train_encodings

{'input_ids': [[101, 2054, 2893, 9436, 1997, 1996, 6092, 2267, 2052, 2941, 2079, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2023, 3232, 4741, 2184, 2086, 2077, 3098, 2037, 5030, 2556, 1517, 2054, 2027, 2179, 2503, 2904, 2037, 3268, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2044, 2611, 1521, 1055, 3611, 7659, 2014, 2606, 2125, 2005, 2893, 5798, 11637, 1010, 3566, 4084, 1999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 21887, 5404, 6762, 2537, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2045, 1521, 1055, 1037, 3492, 4689, 3114, 2339, 2208, 1997, 6106, 2015, 1521, 8292, 22573, 2072, 1998, 22953, 10695, 2024, 2196, 1999, 1037, 3496, 2362, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

###  Turn our labels and encodings into a tf.Dataset object

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),
                                                    list([x[0] for x in y_train.values])))

test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings),
                                                    list([x[0] for x in y_test.values])))

In [12]:
train_dataset 

<TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(49,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(49,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>

## Fine-tuning with native TensorFlow


In [13]:
model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)

optimizerr = tf.keras.optimizers.Adam(learning_rate=5e-5)
losss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) # Computes the crossentropy loss between the labels and predictions. 
model.compile(optimizer=optimizerr,                                     
              loss=losss,
              metrics=['accuracy'])

model.fit(train_dataset.shuffle(len(X_train)).batch(BATCH_SIZE), 
          epochs=N_EPOCHS,
          batch_size=BATCH_SIZE)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


Epoch 1/3
18/18 [==============================] - 26s 95ms/step - loss: 1.1935 - accuracy: 0.5964
Epoch 2/3
18/18 [==============================] - 2s 100ms/step - loss: 0.5020 - accuracy: 0.7786
Epoch 3/3
18/18 [==============================] - 2s 97ms/step - loss: 0.3329 - accuracy: 0.8893


## Model Evaluation

In [14]:
model.evaluate(test_dataset.shuffle(len(X_test)).batch(BATCH_SIZE), return_dict=True, batch_size=BATCH_SIZE)

8/8 [==============================] - 4s 30ms/step - loss: 0.5890 - accuracy: 0.7250


{'accuracy': 0.7250000238418579, 'loss': 0.589028537273407}

## Predict on the different text examples

In [15]:
def predict_proba(text_list, model, tokenizer):
  """
  To get array with predicted probabilities for 0 - instructions, 1- ingredients classes 
  for each paragraph in the list of strings
  :param text_list: list[str]
  :param model: transformers.models.distilbert.modeling_tf_distilbert.TFDistilBertForSequenceClassification
  :param tokenizer: transformers.models.distilbert.tokenization_distilbert.DistilBertTokenizer
  :return res: numpy.ndarray
  """
     
  encodings = tokenizer(text_list, max_length=MAX_LEN, truncation=True, padding=True)
  dataset = tf.data.Dataset.from_tensor_slices((dict(encodings))) 
  preds = model.predict(dataset.batch(1)).logits
  res = tf.nn.softmax(preds, axis=1).numpy()
    
  return res

In [19]:
MAX_LEN = 242

Let's do some test cases

In [20]:
string1 = ["""
            Spider-Man 3: Will Tom Holland Starrer Have Charlie Cox Playing Daredevil?
            """]
predict_proba(string1, model, tokenizer)

array([[0.04763322, 0.9523668 ]], dtype=float32)

The result of the predictive function gives an array of arrays. Each inner array contains probability for 0 and 1 classes (i.e. for summary S and question Q). This example is a question so that's good...

In [21]:
string2 = ["""
            Mom Takes Selfies in Daughters Dorm, Regrets it. This well-meaning mom's "good" idea quickly took a turn for the worse.
            """]
predict_proba(string2, model, tokenizer)

array([[0.8906171 , 0.10938298]], dtype=float32)

This question was a summary, so it looks pretty good...

### Let's try on the rest of our ~3000 entries
should take ~2min

---



In [47]:
# load data
df = pd.read_excel('/content/drive/MyDrive/Saved-You-A-Click-CS224N/data/title_classification/data_label_qs.xlsx')
dnp = df.to_numpy()
# titles are first column
titlelist = dnp[:,0]
prob_of_S_list = []
prob_of_Q_list = []
for title in titlelist:
  array = predict_proba([title], model, tokenizer)
  prob_of_S_list.append(array[0][0])
  prob_of_Q_list.append(array[0][1])

In [50]:
data_tuples = list(zip(titlelist, prob_of_S_list, prob_of_Q_list))
df = pd.DataFrame(data_tuples, columns=['titles','Prob(S)','Prob(Q)'])
df

,titles,Prob(S),Prob(Q)
0,What Getting Rid of the Electoral College would actually do,0.844891,0.155109
1,This Couple Waited 10 Years Before Opening Their Wedding Present—What They Found Inside Changed Their Lives,0.881833,0.118167
2,"After Girl’s Dad Cuts Her Hair Off For Getting Birthday Highlights, Mom Steps In",0.828889,0.171111
3,Corona beer stops production,0.959647,0.040353
4,There’s a pretty crazy reason why Game Of Thrones’ Cersei and Bronn are never in a scene together,0.063671,0.936329
...,...,...,...
2781,Flying this summer? 8 things to know if you haven't been on a plane for a year,0.061518,0.938482
2782,‘The Simpsons’ predicted this ‘Game of Thrones’ twist 2 years ago,0.653483,0.346517
2783,Was Syracuse man ‘executed’ over this vulgar insult?,0.660879,0.339121
2784,Uber and Lyft drivers reveal the most annoying things that passengers do during rides,0.665485,0.334515


In [51]:
# export classified titles
dnp = df.to_numpy()
np.save('/content/drive/MyDrive/Saved-You-A-Click-CS224N/data/title_classification/classified_titles_numpy.npy', dnp)
df = pd.DataFrame(dnp, columns = ['titles','answer','QorS'])
df.to_pickle("/content/drive/MyDrive/Saved-You-A-Click-CS224N/data/title_classification/classified_titles_pandas.pkl")